In [ ]:
!pip install pandas
!pip install openai

In [11]:
def buildBuildFirstMessage():
    prompt = "I want you to act as a great developer with best practices in Microservices project. I will give you the information of a Microservice during the following messages. I will expect that in each of those messages you can answer as simply as: \"got it\". When I am finished I will let you know and start asking you questions about it."
    
    return prompt

In [12]:
import openai
import pandas as pd

    
def getContextOfMicroserviceUsingControllers(microserviceName, df):
    filtered_df_controller = df[df['classFilePath'].str.contains(microserviceName, case=False) & df['classFilePath'].str.contains('controller', case=False)]
    filtered_df_service = df[df['classFilePath'].str.contains(microserviceName, case=False) & df['classFilePath'].str.contains('service', case=False)]
    filtered_df = pd.concat([filtered_df_controller, filtered_df_service], ignore_index=True)
    filtered_df = filtered_df[~filtered_df['methodName'].str.contains('test', case=False)]
    
    return filtered_df
    
def injectMicroservicesContext(microservicesNames, microToMethods, mode, fileName):
    
    df = pd.read_csv(fileName)
    
    partialDfs = []
    for microservice in microservicesNames:
        tempDf = getContextOfMicroserviceUsingControllers(microservice, df)
        partialDfs.append(tempDf)
    
    filtered_df = pd.concat(partialDfs, ignore_index=True)
    print(len(filtered_df))
    
    
    microserviceMessages = []
    for index, row in filtered_df.iterrows():
        methodName = row["methodName"] 
        sourceCode =  row['methodSourceCode']
        flowCrud = row["flowCrudDescription"]
        classFilePath = row["classFilePath"]
#         print("Method Name: ", classFilePath, methodName, sourceCode, flowCrud)
        
        if str(flowCrud) == 'nan':
            flowCrud = " "
        
        if mode == 'Code':
            #Only Code
            microPrompt = "In the path: " + classFilePath + ". There is a method called " + methodName + " . With this code: " + sourceCode + ". Remember that for this message you must answer only \"got it\". "
        elif mode == 'Flow':
            #Crud/Flow
            microPrompt = "In the path: " + classFilePath + ". There is a method called " + methodName + " . With this Crud and Flow Information: " + flowCrud + ". Remember that for this message you must answer only \"got it\". "
        
        elif mode == "FlowCode":
            All
            microPrompt = "In the path: " + classFilePath + ". There is a method called " + methodName + " . With this description: " + sourceCode + ". Finally, this is information about Crud and the Control Flow (sequence of calls): " + flowCrud + ". Remember that for this message you must answer only \"got it\". "    
        
        
        microserviceMessages.append(microPrompt)
        

    openai.api_key = 'sk-'

    # Start the conversation with a system message
    messages = [{"role": "system", "content": "You are a helpful assistant."}]

    givingContext = False
    firstMessage = True
    pos = 0
    
    while True:
        # Get user input
        if firstMessage:
            firstMessage = False
            givingContext = True
            user_message = buildBuildFirstMessage()

        elif not givingContext:
            user_message = input("User: ")

        else: 
            user_message = microserviceMessages[pos]

            pos += 1    

            if pos == len(microserviceMessages):
                givingContext = False
        
#         messages = [{"role": "system", "content": "You are a helpful assistant."}]
        
        print("USER MESSAGE: ", user_message)


        # Append the user message to the conversation
        messages.append({"role": "user", "content": user_message})

        # Generate a response
        response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo-16k",
          messages=messages
        )
        
        if givingContext == False:
            # The assistant's reply
            assistant_message = response['choices'][0]['message']['content']
            print("Assistant: ", assistant_message)
    #         dicMethodToSummChatGPT[methodName] = assistant_message

            #Append the assistant message to the conversation
            messages.append({"role": "assistant", "content": assistant_message})

In [15]:
dic = {
#         "ts-admin-basic-info-service": [],
#         "ts-contacts-service": [],
#         "ts-user-service": [],
#         "ts-preserve-service": [],
        "ts-food-service" : [],
#         "ts-price-service" : [],
#         "ts-preserve-other-service" : [],
#         "ts-cancel-service": [],
#         "ts-travel-service": [],
#         "ts-travel2-service": [],
#         "ts-station-service": [],
#         "ts-food-map-service": [],
#         "ts-ticketinfo-service": [],
#         "ts-basic-service": [],
#         "ts-route-service": [],
#         "ts-order-other-service": [],
      }


#Possible values:
#Code: Only the Code is used
#Flow: Only the PO-CCG is used
#FlowCode: Both are used
mode = 'Code'
fileName='outFixed.csv' #PutHere the filename of the path and filename of the dataset csv

In [ ]:
 injectMicroservicesContext(list(dic.keys()), dic, mode, fileName)